In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

## Load Data

In [2]:
account = pd.read_csv('data/account.csv', encoding='ISO-8859-1')
concerts_1415=pd.read_csv('data/concerts_2014-15.csv')
concerts=pd.read_csv('data/concerts.csv')
sample_submission=pd.read_csv('data/sample_submission.csv')
subscriptions=pd.read_csv('data/subscriptions.csv')
test=pd.read_csv('data/test.csv')
tickets_all=pd.read_csv('data/tickets_all.csv')
train=pd.read_csv('data/train.csv')
zipcodes=pd.read_csv('data/zipcodes.csv')

#### Merge Subscription and Construct Train Data

In [53]:
# Handle subscription data
def handle_subscription(group):
    # Safely handle NaN values in cities
    shipping_city = group['shipping.city'].fillna("")
    billing_city = group['billing.city'].fillna("")

    # If there are any subscription data
    if group['season'].notna().any():
        return pd.DataFrame({

            # Total season:
            'total_season': [group.shape[0]],

            # Package features
            'full_package': [(group['package'] == 'Full').sum()],
            'quartet_package': [(group['package'].isin(['Quartet', 'Quartet A', 'Quartet B', 'Quartet CC'])).sum()],
            'trio_package': [(group['package'].isin(['Trio', 'Trio A', 'Trio B'])).sum()],
            'cyo_package': [(group['package'] == 'CYO').sum()],
            'full_upgrade_package': [(group['package'] == 'Full upgrade').sum()],

            # Seat features
            'total_seats': [group['no.seats'].sum()],

            # Location features
            'location_num': [group['location'].nunique()],
            'location_near_resident_sum': [(group['location'] == shipping_city).sum() if shipping_city.any() else (group['location'] == billing_city).sum()],
            'location_not_resident_num': [group.shape[0] - (group['location'] == shipping_city).sum() if shipping_city.any() else group.shape[0] - (group['location'] == billing_city).sum()],

            # Section features
            'section_type_numbers': [group['section'].nunique()],
            'premium_orchestra_number': [(group['section'] == 'Premium Orchestra').sum()],
            'orchestra_number': [(group['section'].isin(['Orchestra', 'Orchestra Front', 'Orchestra Rear'])).sum()],
            'balcony_number': [(group['section'].isin(['Balcony Front', 'Balcony Rear', 'Balcony', 'Santa Rosa'])).sum()],
            'dress_circle_number': [(group['section'] == 'Dress Circle').sum()],
            'Gallery_number': [(group['section'] == 'Gallery').sum()],
            'Box_number': [(group['section'].isin(['Box', 'Box House Left', 'Box House Right'])).sum()],
            'floor_number': [(group['section'] == 'Floor').sum()],

            # Price features (some seasons do not have price level data using another feature to present it and calculate mean without them)
            'mean_price_level': [-1 if (~group['price.level'].isna()).sum()==0 else group[~group['price.level'].isna()]['price.level'].mean()],

            'non_price_level_subscription': [group['season'].isin(['2002-2003','2003-2004','2004-2005']).sum()],

            # Subscription tier features
            'mean_subscription_tier': [group['subscription_tier'].mean()],
            'multiple_subs_number': [(group['multiple.subs'] == 'yes').sum()],

            # other features
            'account.id': [group['account.id'].iloc[0]],
            'label': [group['label'].iloc[0]],
            'shipping.zip.code': [group['shipping.zip.code'].iloc[0]],
            'billing.zip.code': [group['billing.zip.code'].iloc[0]],
            'shipping_city': [shipping_city.iloc[0]],
            'billing_city': [billing_city.iloc[0]],
            'relationship': [group['relationship'].iloc[0]],
            'amount.donated.2013': [group['amount.donated.2013'].iloc[0]],
            'amount.donated.lifetime': [group['amount.donated.lifetime'].iloc[0]],
            'no.donations.lifetime': [group['no.donations.lifetime'].iloc[0]],
            'first.donated': [group['first.donated'].iloc[0]],
        })
    
    # no subscription data, just fill with -1
    else:
        return pd.DataFrame({
            # Total season:
            'total_season': [-1],

            # Package features
            'full_package': [-1],
            'quartet_package': [-1],
            'trio_package': [-1],
            'cyo_package': [-1],
            'full_upgrade_package': [-1],

            # Seat features
            'total_seats': [-1],

            # Location features
            'location_num': [-1],
            'location_near_resident_sum': [-1],
            'location_not_resident_num': [-1],

            # Section features
            'section_type_numbers': [-1],
            'premium_orchestra_number': [-1],
            'orchestra_number': [-1],
            'balcony_number': [-1],
            'dress_circle_number': [-1],
            'Gallery_number': [-1],
            'Box_number': [-1],
            'floor_number': [-1],

            # Price features
            'mean_price_level': [-1],

            'non_price_level_subscription': [-1],

            # Subscription tier features
            'mean_subscription_tier': [-1],
            'multiple_subs_number': [-1],

            # other features
            'account.id': [group['account.id'].iloc[0]],
            'label': [group['label'].iloc[0]],
            'shipping.zip.code': [group['shipping.zip.code'].iloc[0]],
            'billing.zip.code': [group['billing.zip.code'].iloc[0]],
            'shipping_city': [shipping_city.iloc[0]],
            'billing_city': [billing_city.iloc[0]],
            'relationship': [group['relationship'].iloc[0]],
            'amount.donated.2013': [group['amount.donated.2013'].iloc[0]],
            'amount.donated.lifetime': [group['amount.donated.lifetime'].iloc[0]],
            'no.donations.lifetime': [group['no.donations.lifetime'].iloc[0]],
            'first.donated': [group['first.donated'].iloc[0]],
        })


In [54]:
# Merge train with account
train_merged = pd.merge(train, account, on='account.id', how='left')

# Merge train with subscriptions
train_merged = pd.merge(train_merged, subscriptions, on='account.id', how='left')


print(train_merged.head(5))
# Apply the function using groupby
train_merged = train_merged.groupby('account.id',group_keys=False).apply(handle_subscription)

train_merged=train_merged.reset_index(drop=True)

train_merged.head()

        account.id  label shipping.zip.code billing.zip.code shipping.city  \
0  001i000000NuRo3      0               NaN              NaN           NaN   
1  001i000000NuRxd      0               NaN              NaN           NaN   
2  001i000000NuQGN      0               NaN            94597           NaN   
3  001i000000NuPfL      0               NaN            94005           NaN   
4  001i000000NuQkP      0               NaN            94610           NaN   

   billing.city relationship  amount.donated.2013  amount.donated.lifetime  \
0           NaN          NaN                  0.0                      0.0   
1           NaN          NaN                  0.0                      0.0   
2  Walnut Creek          NaN                  0.0                      0.0   
3      Brisbane          NaN                  0.0                      0.0   
4       Oakland          NaN                  0.0                      0.0   

   no.donations.lifetime first.donated     season package  no.

C:\Users\24549\AppData\Local\Temp\ipykernel_22220\1881199233.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_merged = train_merged.groupby('account.id',group_keys=False).apply(handle_subscription)


,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,label,shipping.zip.code,billing.zip.code,shipping_city,billing_city,relationship,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,first.donated
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0,94102,94102,San Francisco,San Francisco,NaN,0.0,0.0,0,NaN
1,11,11,0,0,0,0,20.0,1,11,0,...,1,NaN,94124,,San Francisco,NaN,0.0,0.0,0,NaN
2,21,21,0,0,0,0,39.0,1,0,21,...,1,NaN,94022,,Los Altos,NaN,1000.0,11913.0,20,1991-07-04 00:00:00
3,13,10,3,0,0,0,25.0,2,0,13,...,1,NaN,94563,,Orinda,NaN,300.0,2570.0,15,1997-04-17 00:00:00
4,19,16,0,0,3,0,35.0,1,0,19,...,0,NaN,94618,,Oakland,NaN,500.0,5929.0,26,1989-07-04 00:00:00


#### Missing Values,Duplicates and Outliers



##### Description

In [55]:
train_merged[train_merged['mean_price_level'].isna()]['account.id']

Series([], Name: account.id, dtype: object)

In [56]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   total_season                  6941 non-null   int64  
 1   full_package                  6941 non-null   int64  
 2   quartet_package               6941 non-null   int64  
 3   trio_package                  6941 non-null   int64  
 4   cyo_package                   6941 non-null   int64  
 5   full_upgrade_package          6941 non-null   int64  
 6   total_seats                   6941 non-null   float64
 7   location_num                  6941 non-null   int64  
 8   location_near_resident_sum    6941 non-null   int64  
 9   location_not_resident_num     6941 non-null   int64  
 10  section_type_numbers          6941 non-null   int64  
 11  premium_orchestra_number      6941 non-null   int64  
 12  orchestra_number              6941 non-null   int64  
 13  bal

##### Duplicates

In [57]:
train_without_duplicates=train_merged.drop_duplicates()

##### Missing Values


In [58]:
train_merged.isnull().sum()

total_season                       0
full_package                       0
quartet_package                    0
trio_package                       0
cyo_package                        0
full_upgrade_package               0
total_seats                        0
location_num                       0
location_near_resident_sum         0
location_not_resident_num          0
section_type_numbers               0
premium_orchestra_number           0
orchestra_number                   0
balcony_number                     0
dress_circle_number                0
Gallery_number                     0
Box_number                         0
floor_number                       0
mean_price_level                   0
non_price_level_subscription       0
mean_subscription_tier             0
multiple_subs_number               0
account.id                         0
label                              0
shipping.zip.code               6845
billing.zip.code                 980
shipping_city                      0
b

In [64]:
train_merged['shipping.zip.code']=train_merged['shipping.zip.code'].fillna("")
train_merged['billing.zip.code']=train_merged['billing.zip.code'].fillna("")
train_merged['relationship']=train_merged['relationship'].fillna("")
train_merged['first.donated']=train_merged.apply(lambda x: '1800-01-01 00:00:00' if x['no.donations.lifetime']==0 else (0 if pd.isna(x['first.donated']) else x['first.donated']),axis=1)

train_merged.head(5)

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,label,shipping.zip.code,billing.zip.code,shipping_city,billing_city,relationship,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,first.donated
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0,94102,94102,San Francisco,San Francisco,,0.0,0.0,0,1800-01-01 00:00:00
1,11,11,0,0,0,0,20.0,1,11,0,...,1,,94124,,San Francisco,,0.0,0.0,0,1800-01-01 00:00:00
2,21,21,0,0,0,0,39.0,1,0,21,...,1,,94022,,Los Altos,,1000.0,11913.0,20,1991-07-04 00:00:00
3,13,10,3,0,0,0,25.0,2,0,13,...,1,,94563,,Orinda,,300.0,2570.0,15,1997-04-17 00:00:00
4,19,16,0,0,3,0,35.0,1,0,19,...,0,,94618,,Oakland,,500.0,5929.0,26,1989-07-04 00:00:00


Turn first donated to timestamp data

In [69]:
train_merged['first.donated'] = pd.to_datetime(train_merged['first.donated'])

train_merged.head(5)

,total_season,full_package,quartet_package,trio_package,cyo_package,full_upgrade_package,total_seats,location_num,location_near_resident_sum,location_not_resident_num,...,label,shipping.zip.code,billing.zip.code,shipping_city,billing_city,relationship,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,first.donated
0,-1,-1,-1,-1,-1,-1,-1.0,-1,-1,-1,...,0,94102,94102,San Francisco,San Francisco,,0.0,0.0,0,1800-01-01
1,11,11,0,0,0,0,20.0,1,11,0,...,1,,94124,,San Francisco,,0.0,0.0,0,1800-01-01
2,21,21,0,0,0,0,39.0,1,0,21,...,1,,94022,,Los Altos,,1000.0,11913.0,20,1991-07-04
3,13,10,3,0,0,0,25.0,2,0,13,...,1,,94563,,Orinda,,300.0,2570.0,15,1997-04-17
4,19,16,0,0,3,0,35.0,1,0,19,...,0,,94618,,Oakland,,500.0,5929.0,26,1989-07-04


In [70]:
# check missing values
train_merged.isnull().sum()

total_season                    0
full_package                    0
quartet_package                 0
trio_package                    0
cyo_package                     0
full_upgrade_package            0
total_seats                     0
location_num                    0
location_near_resident_sum      0
location_not_resident_num       0
section_type_numbers            0
premium_orchestra_number        0
orchestra_number                0
balcony_number                  0
dress_circle_number             0
Gallery_number                  0
Box_number                      0
floor_number                    0
mean_price_level                0
non_price_level_subscription    0
mean_subscription_tier          0
multiple_subs_number            0
account.id                      0
label                           0
shipping.zip.code               0
billing.zip.code                0
shipping_city                   0
billing_city                    0
relationship                    0
amount.donated

#### Merge Subscription and Construct Test Data

In [60]:
# Merge test with account
test_merged = pd.merge(test, account, left_on="ID",right_on='account.id', how='left')

# Merge test with subscriptions
test_merged = pd.merge(test_merged, subscriptions, left_on='ID',right_on="account.id", how='left')

# Apply the function using groupby
test_merged = test_merged.groupby('ID').apply(handle_subscription)

test_merged.head(5)

KeyError: 'account.id'